In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math as math
from collections import OrderedDict, defaultdict

##### The code below was used for my testing purpose

In [ ]:
train_features = pd.read_csv("train-features.txt", sep= " ", header=None)
train_labels = pd.read_csv("train-labels.txt", sep= " ", header=None)
test_features = pd.read_csv("test-features.txt", sep= " ", header=None)
test_labels = pd.read_csv("test-labels.txt", sep= " ", header=None)
spmsga1 = pd.read_csv("spmsga1.txt", sep= " ", header=None)

In [ ]:
#Train Data
train_features.columns = ["Msg Number", "Word Counter", "Number of Occurrences"]
train_labels.columns = ["Spam/NotSpam"]

#Test Data
test_features.columns = ["Msg Number", "Word Counter", "Number of Occurrences"]
test_labels.columns = ["Spam/NotSpam"]
test_labels.insert(0, "Msg Number", range(1, 1 + len(test_labels)))

In [ ]:
#print (train_features)
#print (train_labels)

In [ ]:
spam_length = len( train_labels[ train_labels['Spam/NotSpam'] == 1])
ham_length = len( train_labels[ train_labels['Spam/NotSpam'] == 0])

test_length_spam = len( test_labels[ test_labels['Spam/NotSpam'] == 1])
test_length_ham = len( test_labels[ test_labels['Spam/NotSpam'] == 0])

test_PofSpam = (test_length_spam /  test_length_spam + test_length_ham )
test_PofHam = (test_length_ham /  test_length_spam + test_length_ham )

In [ ]:
spam_dict = defaultdict(int)
ham_dict = defaultdict(int)

##### Count the number of occurences of word in Spam and Ham

In [ ]:
for index,row in train_features.iterrows():
    if train_labels["Spam/NotSpam"].values[row["Msg Number"]-1] == 1:
        spam_dict[row["Word Counter"]] += row["Number of Occurrences"]
    else:
        ham_dict[row["Word Counter"]] += row["Number of Occurrences"]

##### Testing weather the msg is Spam or Ham

In [ ]:
result_dict = defaultdict(int)

msg_num = -1
PofSpam = 0
PofHam = 0
for index,row in test_features.iterrows():
    if (msg_num != row["Msg Number"]):
        if PofSpam + math.log(test_PofSpam) > PofHam + math.log(test_PofHam):
            result_dict[msg_num] = 1
        else:
            result_dict[msg_num] = 0
        PofSpam = 0
        PofHam = 0

    #Finding Probability of Spam for Each Word
    #Applied Laplase Smoothing to Avoid 0s
    #Applied Log to manage the underflow of data
    
    #spam_dict['word'] returns number of occurences of word in spam
    #ham_dic['word'] returns number of occurences of word in ham 
    PofXInSpam = ( spam_dict[row["Word Counter"]] + 1 ) / (spam_length + 2 )
    PofXInHam = ( ham_dict[row["Word Counter"]] + 1 ) / (ham_length + 2 )
    
    PofSpam += ( math.log( PofXInSpam ) * row["Number of Occurrences"] )
    PofHam += ( math.log( PofXInHam ) * row["Number of Occurrences"] )

    msg_num = row["Msg Number"]
    
#For the last Result
if PofSpam + math.log(test_PofSpam) > PofHam + math.log(test_PofHam):
    result_dict[msg_num] = 1
else:
    result_dict[msg_num] = 0

##### Converting Result Dictionary into Dataframe

In [ ]:
result_dict.pop(-1)
predicted_result = pd.DataFrame.from_dict(result_dict, orient="index")
predicted_result.columns = ["Spam/NotSpam"]
predicted_result.insert(0, "Msg Number", range(1, 1 + len(predicted_result)))

##### Accuracy

In [ ]:
total_matches = np.sum(predicted_result["Spam/NotSpam"].values == test_labels["Spam/NotSpam"].values)
accuracy = total_matches/len(predicted_result)
accuracy*100

In [ ]:
fig, arr = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=True)

fig.set_figwidth(16)
fig.set_figheight(50)

arr[0].scatter (predicted_result['Spam/NotSpam'],predicted_result['Msg Number'], c='r' )
arr[0].set_title('Predicted Result')
arr[0].set_xlabel('Spam/NotSpam')
arr[0].set_ylabel('Msg Number')

arr[1].scatter (test_labels['Spam/NotSpam'],test_labels['Msg Number'], c='b' )
arr[1].set_title('Actual Result')
arr[1].set_xlabel('Spam/NotSpam')
arr[1].set_ylabel('Msg Number')